**I. ESTABELECENDO A CONEXÃO COM O BANCO DE DADOS**

In [1]:
import psycopg2
import os
import pandas as pd
import re

conn = psycopg2.connect(
    dbname=os.getenv('DB_NAME'),
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD'),
    host=os.getenv('DB_HOST'),
    port=os.getenv('DB_PORT')
)

# Criando um cursor para executar comandos SQL
cur = conn.cursor()

**II. AJUSTE DE DADOS**

**1. Remover Erros nos Registros**

In [2]:
tabelas = {
    'clientes': """
        DELETE FROM public.clientes
        WHERE id_cliente IS NULL 
            OR nome IS NULL 
            OR telefone IS NULL 
            OR email IS NULL 
            OR endereco IS NULL 
            OR cidade IS NULL 
            OR estado IS NULL 
            OR cep IS NULL 
            OR data_nascimento IS NULL 
            OR genero IS NULL 
            OR status_cliente IS NULL;
    """,
    'orcamentos': """
        DELETE FROM public.orcamentos
        WHERE id_orcamento IS NULL 
            OR data_orcamento IS NULL 
            OR valor_total IS NULL 
            OR status_orcamento IS NULL 
            OR id_cliente IS NULL 
            OR id_veiculo IS NULL;
    """,
    'ordens_servico': """
        DELETE FROM public.ordens_servico
        WHERE id_ordem_servico IS NULL 
            OR data_abertura IS NULL 
            OR data_fechamento IS NULL 
            OR status_os IS NULL 
            OR id_funcionario IS NULL 
            OR id_veiculo IS NULL 
            OR id_orcamento IS NULL 
            OR valor_total IS NULL;
    """,
    'veiculos': """
        DELETE FROM public.veiculos
        WHERE id_veiculo IS NULL 
            OR placa IS NULL 
            OR modelo IS NULL 
            OR marca IS NULL 
            OR ano_fabricacao IS NULL 
            OR cor IS NULL 
            OR tipo_combustivel IS NULL 
            OR id_cliente IS NULL;
    """,
    'produtos': """
        DELETE FROM public.produtos
        WHERE id_produto IS NULL 
            OR nome_produto IS NULL 
            OR descricao IS NULL 
            OR preco_custo IS NULL 
            OR preco_venda IS NULL 
            OR id_fornecedor IS NULL;
    """,
    'produtos_orcamento': """
        DELETE FROM public.produtos_orcamento
        WHERE id_produto_orcamento IS NULL 
            OR tipo_item IS NULL 
            OR id_item IS NULL 
            OR quantidade IS NULL 
            OR preco_unitario IS NULL 
            OR id_orcamento IS NULL;
    """,
    'fornecedores': """
        DELETE FROM public.fornecedores
        WHERE id_fornecedor IS NULL 
            OR nome_fornecedor IS NULL 
            OR cnpj IS NULL 
            OR telefone IS NULL 
            OR email IS NULL 
            OR endereco IS NULL 
            OR contato_responsavel IS NULL;
    """,
    'servicos': """
        DELETE FROM public.servicos
        WHERE id_servico IS NULL 
            OR descricao_servico IS NULL 
            OR preco_servico IS NULL 
            OR tempo_estimado IS NULL;
    """,
    'funcionarios': """
        DELETE FROM public.funcionarios
        WHERE id_funcionario IS NULL
            OR nome_funcionario IS NULL
            OR cargo IS NULL
            OR salario IS NULL
            OR data_contratacao IS NULL
            OR data_nascimento IS NULL
            OR genero IS NULL
            OR telefone IS NULL
            OR email IS NULL;
    """
}

# Removendo os registros dos campos nulos para cada tabela
for tabela, query in tabelas.items():
    cur.execute(query)
    conn.commit()
    print(f"Registros com campos nulos removidos da tabela {tabela}")

Registros com campos nulos removidos da tabela clientes
Registros com campos nulos removidos da tabela orcamentos
Registros com campos nulos removidos da tabela ordens_servico
Registros com campos nulos removidos da tabela veiculos
Registros com campos nulos removidos da tabela produtos
Registros com campos nulos removidos da tabela produtos_orcamento
Registros com campos nulos removidos da tabela fornecedores
Registros com campos nulos removidos da tabela servicos
Registros com campos nulos removidos da tabela funcionarios


**2. Padronização dos Campos de Telefone**

In [3]:

def padronizar_telefone(telefone):
    telefone = re.sub(r'\D', '', telefone)
    if len(telefone) == 10:
        return f"({telefone[:2]}) {telefone[2:6]}-{telefone[6:]}"
    elif len(telefone) == 11:
        return f"({telefone[:2]}) {telefone[2:7]}-{telefone[7:]}"
    return telefone

tabelas_telefones = {
    'clientes': 'id_cliente',
    'fornecedores': 'id_fornecedor',
    'funcionarios': 'id_funcionario'
}

# Atualização de telefones nas tabelas relevantes
for tabela, id_coluna in tabelas_telefones.items():
    cur.execute(f"SELECT {id_coluna}, telefone FROM public.{tabela};")
    registros = cur.fetchall()
    
    for id_registro, telefone in registros:
        telefone_padronizado = padronizar_telefone(telefone)
        cur.execute(f"""
            UPDATE public.{tabela}
            SET telefone = %s
            WHERE {id_coluna} = %s;
        """, (telefone_padronizado, id_registro))

conn.commit()


**3. Formatação dos Campos CPF/CNPJ**

In [4]:
def formatar_cpf(cpf):
    cpf = re.sub(r'\D', '', cpf)
    return f"{cpf[:3]}.{cpf[3:6]}.{cpf[6:9]}-{cpf[9:]}"

def formatar_cnpj(cnpj):
    cnpj = re.sub(r'\D', '', cnpj)
    return f"{cnpj[:2]}.{cnpj[2:5]}.{cnpj[5:8]}/{cnpj[8:12]}-{cnpj[12:]}"

# Atualização de CNPJs na tabela fornecedores
cur.execute("SELECT id_fornecedor, cnpj FROM public.fornecedores;") 
registros = cur.fetchall()
for id_fornecedor, cnpj in registros:
    cnpj_formatado = formatar_cnpj(cnpj)
    cur.execute("""
        UPDATE public.fornecedores
        SET cnpj = %s
        WHERE id_fornecedor = %s;
    """, (cnpj_formatado, id_fornecedor))
conn.commit()

**4. Padronização de Nomes e Placas de Veículos**

In [5]:
def padronizar_nome(nome):
    return nome.title().strip()

def padronizar_placa(placa):
    return placa.upper().strip()

# Atualização de nomes nas tabelas relevantes
tabelas_nome = ['clientes', 'funcionarios', 'fornecedores', 'produtos']
for tabela in tabelas_nome:
    if tabela == 'clientes':
        cur.execute(f"SELECT id_cliente, nome FROM public.{tabela};")
        id_coluna = 'id_cliente'
        nome_coluna = 'nome'
    elif tabela == 'funcionarios':
        cur.execute(f"SELECT id_funcionario, nome_funcionario FROM public.{tabela};")
        id_coluna = 'id_funcionario'
        nome_coluna = 'nome_funcionario'
    elif tabela == 'fornecedores':
        cur.execute(f"SELECT id_fornecedor, nome_fornecedor FROM public.{tabela};")
        id_coluna = 'id_fornecedor'
        nome_coluna = 'nome_fornecedor'
    elif tabela == 'produtos':
        cur.execute(f"SELECT id_produto, nome_produto FROM public.{tabela};")
        id_coluna = 'id_produto'
        nome_coluna = 'nome_produto'
    
    registros = cur.fetchall()
    for registro in registros:
        id_item, nome = registro
        nome_formatado = padronizar_nome(nome)
        cur.execute(f"""
            UPDATE public.{tabela}
            SET {nome_coluna} = %s
            WHERE {id_coluna} = %s;
        """, (nome_formatado, id_item))
    conn.commit()  

# Atualização de placas de veículos
cur.execute("SELECT id_veiculo, placa FROM public.veiculos;")
registros = cur.fetchall()
for registro in registros:
    id_veiculo, placa = registro
    placa_formatada = padronizar_placa(placa)
    cur.execute("""
        UPDATE public.veiculos
        SET placa = %s
        WHERE id_veiculo = %s;
    """, (placa_formatada, id_veiculo))
conn.commit() 

**III. EXPORTAÇÃO DE DADOS**

In [6]:
def exportar_tabela(nome_tabela, arquivo):
    cur.execute(f"SELECT * FROM public.{nome_tabela};")
    dados = cur.fetchall()
    colunas = [desc[0] for desc in cur.description]
    df = pd.DataFrame(dados, columns=colunas)
    df.to_excel(arquivo, index=False)

exportar_tabela('clientes', 'clientes_tratados.xlsx')
exportar_tabela('fornecedores', 'fornecedores_tratados.xlsx')
exportar_tabela('veiculos', 'veiculos_tratados.xlsx')
exportar_tabela('produtos', 'produtos_tratados.xlsx')
exportar_tabela('produtos_orcamento', 'produtos_orcamento_tratados.xlsx')
exportar_tabela('ordens_servico', 'ordens_servico_tratados.xlsx')
exportar_tabela('orcamentos', 'orcamentos_tratados.xlsx')
exportar_tabela('servicos', 'servicos_tratados.xlsx')
exportar_tabela('funcionarios', 'funcionarios_tratados.xlsx')


cur.close()
conn.close()